In [2]:
#from geojson import FeatureCollection
import geopandas


In [3]:
url_coral_reef = "https://opendata.arcgis.com/datasets/9229d814438349948c99b5e61a084418_7.geojson"

#my_feature = FeatureCollection(url_coral_reef)


In [7]:

from geojson import Feature, Polygon, FeatureCollection
# filter_attribtues = ['objectid','sourcethm','geometry']
gdf = geopandas.read_file(url_coral_reef)
gdf
#geopandas.GeoDataFrame.from_features(url_coral_reef['features'])
#gdf.explore("sourcethm", legend=True)
#print(gdf["objectid","sourcethm","geometry"])

,objectid,id,acres,chart1,chart2,notes,sourcethm,createdby,agency,publishdate,...,createdate,modifieddate,modifiedby,deliverydate,sourceid,severity,comments,st_areashape,st_perimetershape,geometry
0,1,0,50.719,19320_1,,,Coral reefs_hawa,,,1899-12-30T00:00:00+00:00,...,None,None,None,2016-09-13T00:00:00+00:00,None,None,None,205258.698180,2627.625559,"POLYGON ((-155.89825 20.23193, -155.89826 20.2..."
1,2,0,33.739,19320_1,,,Coral reefs_hawa,,,1899-12-30T00:00:00+00:00,...,None,None,None,2016-09-13T00:00:00+00:00,None,None,None,136540.403761,1660.745763,"POLYGON ((-155.90107 20.22022, -155.90108 20.2..."
2,3,0,24.451,19320_1,,,Coral reefs_hawa,,,1899-12-30T00:00:00+00:00,...,None,None,None,2016-09-13T00:00:00+00:00,None,None,None,98952.901967,1339.625457,"POLYGON ((-155.90286 20.21338, -155.90282 20.2..."
3,4,0,78.146,19320_1,,,Coral reefs_hawa,,,1899-12-30T00:00:00+00:00,...,None,None,None,2016-09-13T00:00:00+00:00,None,None,None,316255.862683,3513.999694,"POLYGON ((-155.90131 20.18303, -155.90127 20.1..."
4,5,0,90.205,19320_1,,,Coral reefs_hawa,,,1899-12-30T00:00:00+00:00,...,None,None,None,2016-09-13T00:00:00+00:00,None,None,None,365061.336994,3639.728460,"POLYGON ((-155.89961 20.16625, -155.89960 20.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,470,0,0.000,19350_1,,,Coral reefs_maui,,,1899-12-30T00:00:00+00:00,...,None,None,None,2016-09-13T00:00:00+00:00,None,None,None,0.120163,1.674088,"POLYGON ((-156.52241 20.77858, -156.52241 20.7..."
470,471,0,1.664,19347_1,19350_1,,Coral reefs_maui,,,1899-12-30T00:00:00+00:00,...,None,None,None,2016-09-13T00:00:00+00:00,None,None,None,6732.837735,1016.144926,"MULTIPOLYGON (((-156.51905 20.78111, -156.5190..."
471,472,0,0.467,19347_1,19350_1,,Coral reefs_maui,,,1899-12-30T00:00:00+00:00,...,None,None,None,2016-09-13T00:00:00+00:00,None,None,None,1890.686633,484.262692,"MULTIPOLYGON (((-156.51542 20.78764, -156.5155..."
472,473,0,5.018,19347_1,19350_1,,Coral reefs_maui,,,1899-12-30T00:00:00+00:00,...,None,None,None,2016-09-13T00:00:00+00:00,None,None,None,20309.512520,775.651696,"POLYGON ((-156.49815 20.79667, -156.49861 20.7..."


In [6]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
import pyproj 

pyproj.datadir.set_data_dir('C:\\Users\\johnk\\anaconda3\\envs\\ocean_haz\\Library\\share\\proj')  

In [2]:
from flask import Flask, render_template
import sqlite3
import psycopg2 
import osgeo.ogr
import shapely
import shapely.wkt 
import geopandas as gpd 

app = Flask(__name__)

def get_db_connection():
    conn = sqllite3.connect('geotech_ocean_haz.db')
    conn.row_factory = sqlite3.Row
    return conn 

@app.route('/')
def index():
    conn= get_db_connection()
    posts = conn.execute('SELECT * FROM coral_reefs').fetchall()
    conn.close() 
    return render_template('index.html', posts = posts)

connection = psycopg2.connect(database="geotech_ocean_haz", user="postgres", password = "postgres")
cursor = connection.cursor()

cursor.execute("SELECT objectid, acres, featureuid FROM coral_reefs")



In [3]:
rows_list=[]
for objectid,acres,featureuid in cursor:
    data= {'OBJECTID':objectid, 'ACRES':acres, 'FEATUREUID':featureuid}
    rows_list.append(data)
gdf=gpd.GeoDataFrame(rows_list, crs='epsg:26904').set_index('OBJECTID')
gdf.head()


C:\Users\johnk\AppData\Local\Temp/ipykernel_20284/1899271157.py:5: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  gdf=gpd.GeoDataFrame(rows_list, crs='epsg:26904').set_index('OBJECTID')


,ACRES,FEATUREUID
OBJECTID,,
1,50.719,{ECC85DBE-78B8-4614-A0AD-C43C8D3ED3EC}
2,33.739,{3ABCABAA-544D-41C5-AF4F-536755B1AD6A}
3,24.451,{FF0E9110-A3BE-49E2-A05E-2AECE32E4953}
4,78.146,{733E2304-87B1-4670-AEA5-909B31331BB4}
5,90.205,{7B347FB5-0F92-46CD-BAE8-C5A34F27FDA5}


In [4]:
print ('the coral reef in has an acreage of' + str(gdf.ACRES.sum())

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (Temp/ipykernel_20284/4102232325.py, line 1)

In [ ]:
gdf.plot()